In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'TRUESlatewiseGRU'
device = 'cuda:2'
seed = 1337
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SessionwiseGRU(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        slate_size = shp[-1]
        # item_embs: batch, session, slate, embedding
        # user_embs: batch, session, embedding
        item_embs, user_embs = self.embedding(batch)
        
        item_embs = item_embs.flatten(0, 1)
        hidden =  user_embs.flatten(0,1)[None, ...].contiguous()

        rnn_out, _ = self.rnn_layer(
            item_embs,
            hidden,
        )
        return self.out_layer(rnn_out).reshape(shp)

In [4]:
a = torch.arange(30).reshape(2,3,5)
a.shape

torch.Size([2, 3, 5])

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [5]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SessionwiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 2879.38it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.0, 'roc-auc': 1.0, 'accuracy': 0.75}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.45
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 
Val update: epoch: 1 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.46
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SessionwiseGRU(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (rnn_layer): GRU(32, 32, batch_first=True, dropout=0.1)
   (out_layer): Linear(in_features=32, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [6]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [7]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings
        ),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating TRUESlatewiseGRU with svd embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.0, 'roc-auc': 0.44752010703086853, 'accuracy': 0.8990721106529236}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.4337197244167328 | f1: 0.2125655561685562 | auc: 0.6119330525398254 | treshold: 0.13
Test: accuracy: 0.4342195689678192 | f1: 0.2128186672925949 | auc: 0.609704852104187 | 
Val update: epoch: 1 |accuracy: 0.7074616551399231 | f1: 0.22924356162548065 | auc: 0.6173497438430786 | treshold: 0.13
Test: accuracy: 0.7068232893943787 | f1: 0.22689075767993927 | auc: 0.6124050617218018 | 
Val update: epoch: 2 |accuracy: 0.7347403764724731 | f1: 0.23202745616436005 | auc: 0.6306021809577942 | treshold: 0.12
Test: accuracy: 0.7327699065208435 | f1: 0.22610269486904144 | auc: 0.6224696040153503 | 
Val update: epoch: 3 |accuracy: 0.7102633714675903 | f1: 0.23810872435569763 | auc: 0.6441982984542847 | treshold: 0.12
Test: accuracy: 0.7069638967514038 | f1: 0.22862781584262848 | auc: 0.6319550275802612 | 
Val update: epoch: 4 |accuracy: 0.699040949344635 | f1: 0.24546507000923157 | auc: 0.6534899473190308 | treshold: 0.13
Test: accuracy: 0.6970444917678833 | f1: 0.2

/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.17243482172489166, 'roc-auc': 0.5262157917022705, 'accuracy': 0.660834789276123}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.37526747584342957 | f1: 0.20583170652389526 | auc: 0.627719521522522 | treshold: 0.12
Test: accuracy: 0.3800143599510193 | f1: 0.20591825246810913 | auc: 0.6218335628509521 | 
Val update: epoch: 1 |accuracy: 0.8291691541671753 | f1: 0.24402207136154175 | auc: 0.6715437173843384 | treshold: 0.15000000000000002
Test: accuracy: 0.8294644951820374 | f1: 0.24559463560581207 | auc: 0.6640679836273193 | 
Val update: epoch: 2 |accuracy: 0.8528456687927246 | f1: 0.2590496838092804 | auc: 0.701005220413208 | treshold: 0.14
Test: accuracy: 0.853083610534668 | f1: 0.25880685448646545 | auc: 0.6903328895568848 | 
Val update: epoch: 3 |accuracy: 0.8659154176712036 | f1: 0.26386070251464844 | auc: 0.713590145111084 | treshold: 0.14
Test: accuracy: 0.8668457865715027 | f1: 0.26668959856033325 | auc: 0.7048497796058655 | 
Val update: epoch: 4 |accuracy: 0.8726889491081238 | f1: 0.27059444785118103 | auc: 0.7221726179122925 | treshold: 0.16
Test: accuracy: 0.87256312370

In [8]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [9]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [10]:
for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating TRUESlatewiseGRU with explicit embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.2676520347595215, 'roc-auc': 0.43421342968940735, 'accuracy': 0.3817676305770874}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7791321873664856 | f1: 0.8349717855453491 | auc: 0.8269665241241455 | treshold: 0.43
Test: accuracy: 0.7721920013427734 | f1: 0.8283627033233643 | auc: 0.8236024379730225 | 
Val update: epoch: 1 |accuracy: 0.7724568247795105 | f1: 0.8476042151451111 | auc: 0.8690252304077148 | treshold: 0.41000000000000003
Test: accuracy: 0.7645508646965027 | f1: 0.841228187084198 | auc: 0.8673510551452637 | 
Val update: epoch: 2 |accuracy: 0.8170560598373413 | f1: 0.8595096468925476 | auc: 0.8870288133621216 | treshold: 0.47000000000000003
Test: accuracy: 0.8150888681411743 | f1: 0.8563863039016724 | auc: 0.8864738941192627 | 
Val update: epoch: 3 |accuracy: 0.8222802877426147 | f1: 0.8724615573883057 | auc: 0.9022289514541626 | treshold: 0.42000000000000004
Test: accuracy: 0.8184983730316162 | f1: 0.8686246275901794 | auc: 0.9018782377243042 | 
Val update: epoch: 4 |accuracy: 0.825497031211853 | f1: 0.8741386532783508 | auc: 0.9056365489959717 | treshold: 0.54
Test: 

...

Test before learning: {'f1': 0.6257887482643127, 'roc-auc': 0.49290725588798523, 'accuracy': 0.528207004070282}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.777003824710846 | f1: 0.837870180606842 | auc: 0.8240405321121216 | treshold: 0.43
Test: accuracy: 0.7653971910476685 | f1: 0.8290637731552124 | auc: 0.8172584772109985 | 
Val update: epoch: 1 |accuracy: 0.7622503042221069 | f1: 0.8419512510299683 | auc: 0.8727391958236694 | treshold: 0.4
Test: accuracy: 0.75275057554245 | f1: 0.8346405625343323 | auc: 0.8690912127494812 | 
Val update: epoch: 2 |accuracy: 0.8216272592544556 | f1: 0.8616244792938232 | auc: 0.8971524238586426 | treshold: 0.49
Test: accuracy: 0.8198283314704895 | f1: 0.8590400815010071 | auc: 0.8946620225906372 | 
Val update: epoch: 3 |accuracy: 0.8243844509124756 | f1: 0.8733494877815247 | auc: 0.9056565761566162 | treshold: 0.48000000000000004
Test: accuracy: 0.8163946270942688 | f1: 0.8666748404502869 | auc: 0.9035195112228394 | 
Val update: epoch: 4 |accuracy: 0.8211677074432373 | f1: 0.8726708889007568 | auc: 0.9086782336235046 | treshold: 0.52
Test: accuracy: 0.8145810961723328 | f1

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.712209165096283 | f1: 0.7969765663146973 | auc: 0.7191197872161865 | treshold: 0.41000000000000003
Test: accuracy: 0.7047273516654968 | f1: 0.7905668616294861 | auc: 0.7130021452903748 | 
Val update: epoch: 1 |accuracy: 0.7306389808654785 | f1: 0.8051438927650452 | auc: 0.7578662633895874 | treshold: 0.32
Test: accuracy: 0.7229838967323303 | f1: 0.7986714243888855 | auc: 0.7517884969711304 | 
Val update: epoch: 2 |accuracy: 0.7362260222434998 | f1: 0.8036582469940186 | auc: 0.7726535797119141 | treshold: 0.35000000000000003
Test: accuracy: 0.7303590774536133 | f1: 0.7982413172721863 | auc: 0.7667707204818726 | 
Val update: epoch: 3 |accuracy: 0.7436269521713257 | f1: 0.8058323860168457 | auc: 0.7852849960327148 | treshold: 0.37
Test: accuracy: 0.7380486130714417 | f1: 0.8004568219184875 | auc: 0.7795764207839966 | 
Val update: epoch: 4 |accuracy: 0.7450539469718933 | f1: 0.800499677658081 | auc: 0.7969678044319153 | treshold: 0.42000000000000004
Test: 

In [11]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items